In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv("hr.csv")

In [37]:
##Data Pre-processing

In [2]:
df.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,department,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [3]:
feats = ['department','salary']
df_final = pd.get_dummies(df,columns=feats,drop_first=True)

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
X = df_final.drop(['left'],axis=1).values
y = df_final['left'].values

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [7]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [8]:
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [9]:
classifier = Sequential()

In [10]:
classifier.add(Dense(9, kernel_initializer = "uniform",activation = "relu", input_dim=18))

In [11]:
classifier.add(Dense(1, kernel_initializer = "uniform",activation = "sigmoid"))

In [12]:
classifier.compile(optimizer= "adam",loss = "binary_crossentropy",metrics = ["accuracy"])

In [13]:
classifier.fit(X_train, y_train, batch_size = 10, epochs = 10)

Epoch 1/1
10499/10499 [==============================] - 2s 150us/step - loss: 0.4271 - acc: 0.7985


In [14]:
y_pred = classifier.predict(X_test)

In [15]:
y_pred = (y_pred > 0.5)

In [16]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[3335,  126],
       [ 633,  406]])

In [17]:
new_pred = classifier.predict(sc.transform(np.array([[0.26,0.7 ,3., 238., 6., 0.,0.,0.,0., 0.,0.,0.,0.,0.,1.,0., 0.,1.]])))

In [18]:
new_pred = (new_pred > 0.5)
new_pred

array([[False]])

In [19]:
new_pred = (new_pred > 0.6)
new_pred

array([[False]])

In [20]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

In [21]:
def make_classifier():
    classifier = Sequential()
    classifier.add(Dense(9, kernel_initializer = "uniform", activation = "relu", input_dim=18))
    classifier.add(Dense(1, kernel_initializer = "uniform", activation = "sigmoid"))
    classifier.compile(optimizer= "adam",loss = "binary_crossentropy",metrics = ["accuracy"])
    return classifier

In [22]:
classifier = KerasClassifier(build_fn = make_classifier, batch_size=10, nb_epoch=1)

In [25]:
accuracies = cross_val_score(estimator = classifier,X = X_train,y = y_train,cv = 10,n_jobs = -1)

Epoch 1/1
1050/1050 [==============================] - 0s 92us/step
Epoch 1/1
1050/1050 [==============================] - 0s 107us/step
Epoch 1/1
1050/1050 [==============================] - 0s 150us/step
Epoch 1/1
1050/1050 [==============================] - 0s 124us/step
Epoch 1/1
1050/1050 [==============================] - 0s 128us/step
Epoch 1/1
1050/1050 [==============================] - 0s 148us/step
Epoch 1/1
1050/1050 [==============================] - 0s 191us/step
Epoch 1/1
1050/1050 [==============================] - 0s 146us/step
Epoch 1/1
1050/1050 [==============================] - 0s 155us/step
Epoch 1/1
1049/1049 [==============================] - 0s 163us/step


In [26]:
mean = accuracies.mean()
mean

0.8226506836536149

In [27]:
variance = accuracies.var()
variance

0.0012544400306649228

In [28]:
from keras.layers import Dropout

classifier = Sequential()
classifier.add(Dense(9, kernel_initializer = "uniform", activation = "relu", input_dim=18))
classifier.add(Dropout(rate = 0.1))
classifier.add(Dense(1, kernel_initializer = "uniform", activation = "sigmoid"))
classifier.compile(optimizer= "adam",loss = "binary_crossentropy",metrics = ["accuracy"])

In [29]:
from sklearn.model_selection import GridSearchCV
def make_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(9, kernel_initializer = "uniform", activation = "relu", input_dim=18))
    classifier.add(Dense(1, kernel_initializer = "uniform", activation = "sigmoid"))
    classifier.compile(optimizer= optimizer,loss = "binary_crossentropy",metrics = ["accuracy"])
    return classifier

In [30]:
classifier = KerasClassifier(build_fn = make_classifier)

In [31]:
params = {
    'batch_size':[20,35],
    'epochs':[2,3],
    'optimizer':['adam','rmsprop']
}

In [32]:
grid_search = GridSearchCV(estimator=classifier,
                           param_grid=params,
                           scoring="accuracy",
                           cv=2)

In [33]:
grid_search = grid_search.fit(X_train,y_train)

Epoch 1/2
5249/5249 [==============================] - 1s 174us/step - loss: 0.5764 - acc: 0.7567
Epoch 2/2
5249/5249 [==============================] - 0s 86us/step - loss: 0.4007 - acc: 0.7929
Epoch 1/2
5250/5250 [==============================] - 1s 184us/step - loss: 0.5773 - acc: 0.7691
Epoch 2/2
5250/5250 [==============================] - 0s 68us/step - loss: 0.3980 - acc: 0.7981
Epoch 1/2
5249/5249 [==============================] - 1s 142us/step - loss: 0.5971 - acc: 0.7611
Epoch 2/2
5249/5249 [==============================] - 0s 45us/step - loss: 0.4305 - acc: 0.7954
Epoch 1/2
5250/5250 [==============================] - 1s 145us/step - loss: 0.6050 - acc: 0.7514
Epoch 2/2
5250/5250 [==============================] - 0s 75us/step - loss: 0.4261 - acc: 0.7981
Epoch 1/3
5249/5249 [==============================] - 1s 216us/step - loss: 0.5672 - acc: 0.7622
Epoch 2/3
5249/5249 [==============================] - 0s 83us/step - loss: 0.4117 - acc: 0.7624
Epoch 3/3
5249/5249 [====

In [34]:
best_param = grid_search.best_params_
best_accuracy = grid_search.best_score_

In [35]:
best_param


{'batch_size': 20, 'epochs': 3, 'optimizer': 'adam'}

In [36]:
best_accuracy

0.8260786741594438